In [1]:
# All modules needed
import ZEO
import transaction
from ZEO.ClientStorage import ClientStorage
import ZODB
import BTrees.OOBTree
import persistent
import pandas as pd
from datetime import datetime

In [2]:
# Advanced: You may need learn about inherit persistent class ZODB
# Create class NhanVien
class NhanVien():
    def __init__(self):
        self.Ten = ''
        self.GioiTinh = ''
        self.NgaySinh=''
        self.SDT=''
        self.Email=''

In [3]:
# Create class SanPham
class SanPham():
    def __init__(self):
        self.TenSP=''
        self.SoLuong=''
        self.Gia=''
    def TongGia(self):
        return self.SoLuong*self.Gia

In [4]:
def ConnectLocal():
    # Function ZEO.server() return 2 value: address, stop
    address, stop = ZEO.server(path="./CineSV_Xoa13.fs")
    storage = ClientStorage(address)
    db = ZODB.DB(storage=storage, database_name="Cinesv")
    return address, stop, db

In [5]:
# Requirement hosting on 'mayA'
# See on ConnectDistributed.docx
def ConnectDistributed():
    global db
    print("Starting connect Distributed")
    storage = ClientStorage(('mayA', 49762))
    db = ZODB.DB(storage=storage, database_name="Cinesv", transaction=tm)
    return db

In [6]:
def Insert_Update(db):
    dbroot = db.root()

    print("Created object ChiNhanh")
    cn = ChiNhanh()
    cn.Ten = "Kho phía Bắc"
    cn.DiaChi = "336 Nguyễn Trãi, Thanh Xuân, Hà Nội, Việt Nam"

    # Check exist branch ChiNhanh
    try:
        dbroot.ChiNhanh
    except AttributeError:
        dbroot.ChiNhanh = BTrees.OOBTree.BTree()
        print("Create branch ChiNhanh")

    print("Added object ChiNhanh")
    dbroot.ChiNhanh[0] = cn

    transaction.commit()
    print("Commit Successfully")

In [18]:
def SelectNhanVien(db):
    dbroot = db.root()
    
    # Check exist branch NhanVien
    try:
        dbroot['NhanVien']
    except AttributeError:
        print("Not Found branch NhanVien")
        return

    # Print list NhanVien
    print("Tên".ljust(25), end='')
    print("Giới tính".ljust(10), end='')
    print("Ngày sinh".ljust(12), end='')
    print("SDT".ljust(15), end='')
    print("CCCD".ljust(15), end='')
    print("Email".ljust(25), end='')
    print()
    for x in list(dbroot['NhanVien'].keys()):
        print(dbroot['NhanVien'][x].Ten.ljust(25), end='')
        print(dbroot['NhanVien'][x].GioiTinh.ljust(10), end='')
        print(str(dbroot['NhanVien'][x].NgaySinh).ljust(12), end='')
        print(str(dbroot['NhanVien'][x].SDT).ljust(12), end='')
        # Keys là CCCD
        print(str(x).ljust(13), end='')
        print(dbroot['NhanVien'][x].Email.ljust(25), end='')
        print()

In [8]:
# Insert NhanVien From a file .csv
def InsertNhanVienviaFileCSV(db):
    dbroot = db.root()
    try:
        dbroot['NhanVien']
    except KeyError:
        dbroot['NhanVien'] = persistent.mapping.PersistentMapping()
        print("Created branch NhanVien")

    # Read file csv
    dataset = pd.read_csv("./Data_NhanVien.csv")
    print("Starting add data NhanVien")

    for x in range(len(dataset)):
        # Create each object NhanVien
        nv = NhanVien()
        nv.Ten = dataset.iloc[x, 0]
        nv.GioiTinh = dataset.iloc[x, 1]
        nv.NgaySinh = datetime.strptime(dataset.iloc[x, 2], '%m/%d/%Y').date()
        nv.SDT = dataset.iloc[x, 3]
        nv.Email = dataset.iloc[x, 5]

        # Add object into ZODB
        dbroot['NhanVien'][dataset.iloc[x, 4]] = nv

    transaction.commit()
    print("Commit Successfully")

In [9]:
def SelectSanPham(db):
    dbroot = db.root()
    # Check exist SanPHam
    try:
        dbroot.SanPham
    except AttributeError:
        print("Not Found SanPham")
        return

    # Print list SanPham
    print("Mã sản phẩm".ljust(12), end='')
    print("Tên sản phẩm".ljust(25), end='')
    print("Số lượng".ljust(10), end='')
    print("Giá".ljust(13), end='')
    print()
    for x in list(dbroot.SanPham.keys()):
        print(str(x).ljust(12), end='')
        print(dbroot.SanPham[x].TenSP.ljust(25), end='')
        print(str(dbroot.SanPham[x].SoLuong).ljust(10), end='')
        print(str(dbroot.SanPham[x].Gia).ljust(13), end='')
        print()

In [10]:
# Insert SanPham from file .csv
def InsertSanPhamviaFileCSV(db):
    dbroot = db.root()
    try:
        dbroot.SanPham
    except AttributeError:
        dbroot.SanPham = BTrees.OOBTree.BTree()
        print("Created branch SanPham")

    # Read file .csv
    dataset = pd.read_csv("./Data_SanPham.csv")
    print("Bắt đầu thêm dữ liệu vào")

    for x in range(len(dataset)):
        # Create each SanPham
        sp = SanPham()
        sp.TenSP = dataset.iloc[x, 1]
        sp.SoLuong = dataset.iloc[x, 2]
        sp.Gia = dataset.iloc[x, 3]

        # Add SanPham into ZODB
        dbroot.SanPham[dataset.iloc[x, 0]] = sp

    transaction.commit()
    print("Commit Successfully")

In [11]:
def InsertDemo(db):
    dbroot = db.root()

    # Testing multi multi branch in ZODB
    # All branch must start from root (db.root())
    print("Thêm đối tượng vào nhánh vừa tạo")
    dbroot[1] = persistent.mapping.PersistentMapping()
    dbroot[1][1] = persistent.mapping.PersistentMapping()
    dbroot[1][1][1] = persistent.mapping.PersistentMapping()
    dbroot[1][1][1].ChiNhanh = BTrees.OOBTree.BTree()
    dbroot[1][1][1].ChiNhanh[0] = persistent.mapping.PersistentMapping()
    dbroot[1][1][1].ChiNhanh[0][0] = "Intro ZODB"
    dbroot[1][1][1].ChiNhanh[1] = "I am Hoàng Đình Phú"
    
    # You can store anything you need but you need renember 
    #        Structure: number; string; class: properties, method,...; 
    #        Address: Example: dbroot[1][1][1].ChiNhanh[1]

    transaction.commit()
    print("Commit Successfully")

In [12]:
def SelectDemo(db):
    # Kết nối đến db
    # connect = Connect().open()
    connect = ConnectLocal()
    dbroot = connect.open().root()

    print(dbroot[1][1][1].ChiNhanh[1])

In [13]:
def Delete(db):
    dbroot = db.root()
    # Delete anything you need
    del dbroot[1][1][1].ChiNhanh[1]
    transaction.commit()
    print("Commit Successfully")

In [14]:
## Choose method connection
# Connect Distributed
# db = ConnectDistributed()

# Connect Local
address, stop, db = ConnectLocal()
print(address)

dbo = db.open()

('127.0.0.1', 55477)


In [19]:
# Method you need run
InsertSanPhamviaFileCSV(dbo)
SelectSanPham(dbo)

InsertNhanVienviaFileCSV(dbo)
SelectNhanVien(dbo)

Bắt đầu thêm dữ liệu vào
Commit Successfully
Mã sản phẩm Tên sản phẩm             Số lượng  Giá          
1           iPhone XR                184       10000000     
2           iPhone XS                414       13000000     
3           iPhone XS Max            136       16000000     
4           iPhone 11                431       19000000     
5           iPhone 11 Pro            468       22000000     
6           iPhone 11 Pro Max        494       25000000     
7           iPhone 12 Mini           360       28000000     
8           iPhone 12                314       31000000     
9           iPhone 12 Pro            298       34000000     
10          iPhone 12 Pro Max        216       37000000     
11          iPhone 13 Mini           459       40000000     
12          iPhone 13                466       43000000     
13          iPhone 13 Pro            313       46000000     
14          iPhone 13 Pro Max        62        49000000     
Starting add data NhanVien
Commit Succes

In [16]:
# Close connection local (maybe error on jupyter notebook)
# dbo.close()
# stop
# print('Disconnected')